In [ ]:
import torch
import torchvision
from torchvision import transforms
from PIL import Image
import os
import glob, shutil
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import imageio
import time
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Load google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import data
!mkdir Hurricane_Harvey 
!mkdir Hurricane_Harvey/rasters Hurricane_Harvey/vectors 
!gsutil -m cp -n -r gs://geoengine-dataset-houston-uav/rasters/raw Hurricane_Harvey/rasters/ 
!gsutil -m cp -n -r gs://geoengine-dataset-houston-uav/vectors/random-split-_2022_11_17-22_35_45/ Hurricane_Harvey/vectors/

In [ ]:
# Save data in a dataframe and take a look
# df_train = pd.read_csv('/content/Hurricane_Harvey/vectors/random-split-_2022_11_17-22_35_45/CSV/train.csv', index_col = 0)
# Let's see the labels
# df_train['label'].value_counts()
# Let's see how many labels there are
# len(df_train['label'].value_counts())
# you lied in the description, there are more than 25 there are 27!!!!
# The info get automaticaly saved in a raw and in masks folder, so let's load the path
images_path = '/content/Hurricane_Harvey/rasters/raw'
masks_path = '/content/Hurricane_Harvey/vectors/random-split-_2022_11_17-22_35_45/Masks'

# Let's check we have the right amount of pictures
images_filenames = list(sorted(os.listdir(images_path)))
masks_filenames=list(sorted(os.listdir(masks_path)))
# We should have 374
print(len(images_filenames))
# We should have 299
print(len(masks_filenames))

374
299


In [ ]:
# We are going to move the images from a local folder to a folder in Drive, so here we set the folders
# This is for the trainig images (the ones that also have masks)
entrena_imagenes='/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto_yo/Fotos_1/Entrena_fotos_1'
entrena_masks='/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto_yo/Fotos_1/Entrena_masks_1'
valida_imagenes='/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto_yo/Fotos_1/Testing_fotos_1'

In [ ]:
# Create an auxiliary list with the name of the masks, with no termination
masks_filenames_no_termi=[]
for nom in masks_filenames:
  # Extract the name without the .png termination
  masks_filenames_no_termi.append(nom[:-4])
# Let's take a look
masks_filenames_no_termi[0:3]

['10170', '10171', '10172']

In [ ]:
# Here we move the files to their corresponding folder
for dir in images_filenames:
  if dir[:-4] in masks_filenames_no_termi:
    # the png are the masks
    shutil.copy2(masks_path+'/'+dir[:-4]+'.png', entrena_masks) #put the masks in the train folder
    # the tif are the imges
    shutil.copy2(images_path+'/'+dir[:-4]+'.tif', entrena_imagenes) #put the images in the train folder
  else:
    # these are the images for the validation
    shutil.copy2(images_path+'/'+dir[:-4]+'.tif', valida_imagenes) #put just the img in the validation folder

# Check if it did the job
# the images (they have to be 299)
print(len(list(sorted(os.listdir(entrena_imagenes)))))
# the labels (they have to be 299)
print(len(list(sorted(os.listdir(entrena_masks)))))
# the ones we'll have to predict (they have to be 75=374-299)
print(len(list(sorted(os.listdir(valida_imagenes)))))
#  Looks good

299
299
75


In [ ]:
! pip install patchify
import cv2
from patchify import patchify
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = MinMaxScaler()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
alto=256
ancho=256
# imagenes_transf= transforms.Compose([transforms.ToTensor()])
imagenes_transf= transforms.Compose([transforms.Resize([alto,alto]),transforms.ToTensor()])
masks_transf=transforms.Compose([transforms.Resize([alto,alto], interpolation=T.InterpolationMode.BILINEAR), transforms.ToTensor()])

In [ ]:
# select images to make an augmentation
# we are going to select 45 images to make them a transformation
import random
# masks_filenames_no_termi
rotacion=random.sample(masks_filenames_no_termi, 15)
# main_list = list(set(list_2) - set(list_1))
flip_horizontal=random.sample(list(set(masks_filenames_no_termi)-set(rotacion)),15)
# list(set(masks_filenames_no_termi)-set(rotacion))
restante=list(set(masks_filenames_no_termi)-set(rotacion))
flip_vertical=random.sample(list(set(restante)-set(flip_horizontal)),15)

In [ ]:
# patch_size=256
# image_dataset = []
# for foto in masks_filenames_no_termi:
tensor_image_path='/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto_yo/Fotos_1/Tensors/Image_tensors'
tensor_mask_path='/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto_yo/Fotos_1/Tensors/Masks_tensors'
for foto in masks_filenames_no_termi:

  # open and transform the images
  abre_imagen=imagenes_transf(Image.open(entrena_imagenes+'/'+foto+'.tif'))
  abre_mask=imagenes_transf(Image.open(entrena_masks+'/'+foto+'.png'))*255
  abre_mask=abre_mask.int()

  # save them as tensors
  torch.save(abre_imagen, tensor_image_path+'/'+foto+'.pt')
  torch.save(abre_mask, tensor_mask_path+'/'+foto+'.pt')

  # save transformations as tensors also
  if foto in rotacion:
    t_rotacion = T.RandomRotation(degrees=(0, 180))
    imagen_rotada= t_rotacion(abre_imagen)
    mask_rotada= t_rotacion(abre_mask)

    # save the rotated images and masks with a different name
    torch.save(imagen_rotada, tensor_image_path+'/'+foto+'_rotada.pt')
    torch.save(mask_rotada, tensor_mask_path+'/'+foto+'_rotada.pt')
  
  # save horizontal flips
  elif foto in flip_horizontal:
    t_horizontal=T.RandomHorizontalFlip(p=1)
    imagen_hori = t_horizontal(abre_imagen)
    mask_hori = t_horizontal(abre_mask)

    # save the horizontal images and masks with a different name
    torch.save(imagen_hori, tensor_image_path+'/'+foto+'_hori.pt')
    torch.save(mask_hori, tensor_mask_path+'/'+foto+'_hori.pt')
  
  # save vertical flips
  elif foto in flip_vertical:
    t_vertical=T.RandomVerticalFlip(p=1)
    imagen_verti=t_vertical(abre_imagen)
    mask_verti=t_vertical(abre_mask)

    # save the vertical images and masks with a different name
    torch.save(imagen_verti, tensor_image_path+'/'+foto+'_verti.pt')
    torch.save(mask_verti, tensor_mask_path+'/'+foto+'_verti.pt')
  
  else:
    pass

In [ ]:
tensor_image_path='/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto_yo/Fotos_1/Tensors/Image_tensors'
tensor_mask_path='/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto_yo/Fotos_1/Tensors/Masks_tensors'

In [ ]:
# they should be 344 = 299+45 (15 rotated, 15 flipped horizontally, 15 flipped vertically)
print(len(sorted(glob.glob(tensor_image_path+'/*'))))
print(len(sorted(glob.glob(tensor_mask_path+'/*'))))

In [ ]:
# now we have to make our selection of images for the validation and the training dataset
# make the sample of fotos for trianing and validation
fotos_entrena, fotos_valida=train_test_split(sorted(os.listdir(tensor_image_path)), test_size=0.10, random_state=13)

In [ ]:
len(fotos_valida)

In [ ]:
import shutil
tensor_image_path='/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto_yo/Fotos_1/Tensors/Image_tensors'
tensor_mask_path='/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto_yo/Fotos_1/Tensors/Masks_tensors'

In [ ]:
# Folder for training
tensor_image_train='/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto_yo/Fotos_1/Tensors/Tensor_entrenamiento/Tensor_entrenamiento_images'
tensor_mask_train='/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto_yo/Fotos_1/Tensors/Tensor_entrenamiento/Tensor_entrenamiento_masks'

# Folde for validation
tensor_image_val='/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto_yo/Fotos_1/Tensors/Tensor_validacion/Tensor_validacion_images'
tensor_mask_val='/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto_yo/Fotos_1/Tensors/Tensor_validacion/Tensor_validacion_masks'

In [ ]:
for archivo in sorted(os.listdir(tensor_image_path)):
  # print(tensor_image_path+'/'+archivo)
  if archivo in fotos_entrena:
    
    # copy image for training
    shutil.copyfile(tensor_image_path+'/'+archivo,tensor_image_train+'/'+archivo)
    # copy mask for training
    shutil.copyfile(tensor_mask_path+'/'+archivo,tensor_mask_train+'/'+archivo)
  
  # the else is for the tensor that are not in the training, so they are in validation
  else:

    # copy image for validation
    shutil.copyfile(tensor_image_path+'/'+archivo,tensor_image_val+'/'+archivo)
    # copy mask for validation
    shutil.copyfile(tensor_mask_path+'/'+archivo,tensor_mask_val+'/'+archivo)
  

In [ ]:
# check if we have the images in the paths

#  258
print(len(os.listdir(tensor_image_train)))
print(len(os.listdir(tensor_mask_train)))
# 86
print(len(os.listdir(tensor_image_val)))
print(len(os.listdir(tensor_mask_val)))


258
258
86
86


# Data loaders

In [ ]:
# Create a custom Dataset class
class MiTensorDataset(Dataset):
       
    def __init__(self, images:list, masks:list):
        self.image_links = images
        self.mask_links  = masks

    def __len__(self):
        return len(self.image_links) 
        

    def __getitem__(self, index):
        
        # Select a specific image's link
        img_id  = self.image_links[index]
        mask_id = self.mask_links[index]

        # Load the image (as tensor)
        img  = torch.load(img_id)
        mask = torch.load(mask_id)
                      
        # Squeeze mask from (1, H, W) to (H, W) -> This is because loss function accepts (B, H, W)
        mask = mask.squeeze(0)

        # Turn on gradient for image
        img = img.detach().clone().requires_grad_(True)
        mask = mask.long()
        
        return img, mask
    

In [ ]:
# Folder for training
tensor_image_train=sorted(glob.glob('/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto_yo/Fotos_1/Tensors/Tensor_entrenamiento/Tensor_entrenamiento_images/*'))
tensor_mask_train=sorted(glob.glob('/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto_yo/Fotos_1/Tensors/Tensor_entrenamiento/Tensor_entrenamiento_masks/*'))

# Folder for validation
tensor_image_val=sorted(glob.glob('/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto_yo/Fotos_1/Tensors/Tensor_validacion/Tensor_validacion_images/*'))
tensor_mask_val=sorted(glob.glob('/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto_yo/Fotos_1/Tensors/Tensor_validacion/Tensor_validacion_masks/*'))

In [ ]:
tensor_image_train

In [ ]:
dataset_entrenamiento=MiTensorDataset(tensor_image_train,tensor_mask_train)
dataloader_entrenamiento=DataLoader(dataset_entrenamiento, batch_size=10, shuffle=10)
dataset_val=MiTensorDataset(tensor_image_val,tensor_mask_val)
dataloader_val=DataLoader(dataset_val, batch_size=10, shuffle=10)

In [ ]:
for i, batch in enumerate(dataloader_entrenamiento):
    img_batch, img_mask = batch
    print(img_batch.shape)
    print(img_mask.shape)

# UNET

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF
# based on the paper
# https://arxiv.org/abs/1505.04597

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

class UNET(nn.Module):
    def __init__(
            self, in_channels=3, out_channels=1, features=[64, 128, 256, 512],
    ):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNET
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)

In [ ]:
# check to see if the UNET works
x = torch.randn((3, 2, 161, 161))
model = UNET(in_channels=2, out_channels=9)
preds = model(x)

In [ ]:
# Define device
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [ ]:
# UNEt on the data
model = UNET(in_channels=3, out_channels=27).to(device)

In [ ]:
alto=256
ancho=256
# Accuracy
def model_accuracy(predicciones, mask_correcta):#preds, true_mask):
  # this function runs over all the batch, so we have to create a list to save all the accuracy points por each
  # of the images and masks
    lista_accuracy_results= []# initialize list
    # iterate through the predictions
    for pred in range(len(predicciones)):
        # img  = preds[i].to(device)
        # mask = true_mask[i].to(device)
        foto = predicciones[pred]
        mask = mask_correcta[pred]
        
        # here we change the shape of the image, because is has 3 dimensions and we want it to match
        # the dimensions of the mask
        foto = torch.argmax(foto, dim=0)
        # we calculate the accuracy
        acc_num=100*torch.sum(foto == mask).item() / (alto*ancho)
        # we append it to the list
        lista_accuracy_results.append(acc_num)  
        
    return np.mean(lista_accuracy_results)

In [ ]:
# Define a function to calculate the Dice score
def DiceScore(predicciones, mascara_correcta):
    dice_batch = []

    for i in range(len(predicciones)):
        dice_image = []
        img  = predicciones[i].to(device)
        mask = mascara_correcta[i].to(device)
        # img  = predicciones[i]
        # mask = mascara_correcta[i]
        
        img = torch.argmax(img, dim=0)
    
        for label in range(27):
            if torch.sum(mask == label) != 0:
                area_of_intersect = torch.sum((img == label) * (mask == label))
                area_of_img       = torch.sum(img == label)
                area_of_label     = torch.sum(mask == label)
                dice = 2*area_of_intersect / (area_of_img + area_of_label)
                dice_image.append(dice)
        
        dice_batch.append(np.mean([tensor.cpu() for tensor in dice_image]))
    return np.mean(dice_batch)


In [ ]:
epochs = 15
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss().to(device)
total_train_losses   = []
total_train_accuracy=[]
total_val_losses=[]
total_val_accuracy=[]
total_train_dice=[]
total_val_dice=[]

In [ ]:
path_res='/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto_yo/ResUNET_mano/Primero'
for epoch in range(1, epochs+1):
    
    # Train model
    model.train()
    train_losses   = []
    train_accuracy = []
    train_dice       = []
    
    for i, batch in enumerate(dataloader_val):
        
        img_batch, mask_batch = batch   
        img_batch = img_batch.to(device)
        mask_batch = mask_batch.to(device)

        #Train model
        optimizer.zero_grad()
        
        output = model(img_batch)
        loss   = criterion(output, mask_batch)
        loss.backward()

        # torch.nn.utils.clip_grad_norm_(model.parameters(), 6)
        optimizer.step()

        
        dice = DiceScore(output, mask_batch)
        acc = model_accuracy(output, mask_batch)
        train_losses.append(loss.item())
        train_accuracy.append(acc)
        train_dice.append(dice)
        
    
    # RESULTS
    print(f'TRAINING')
    print(f'Epoch: {epoch} | Loss: {np.mean(train_losses):.4f}, Accuracy: {np.mean(train_accuracy):.3f}, Dice: {np.mean(train_dice):.3f}')        
    total_train_losses.append(np.mean(train_losses))
    total_train_accuracy.append(np.mean(train_accuracy))
    total_train_dice.append(np.mean(train_dice))
    
    
    ######################### Validation of the model
    model.eval()
    val_losses   = []
    val_accuracy = []
    val_dice       = []
    
    for i, batch in enumerate(dataloader_val):
        
        img_batch, mask_batch = batch
        img_batch = img_batch.to(device)
        mask_batch = mask_batch.to(device)

        
        with torch.cuda.amp.autocast():
          output = model(img_batch)
          loss   = criterion(output, mask_batch)

        dice = DiceScore(output, mask_batch)
        acc = model_accuracy(output, mask_batch)
        val_losses.append(loss.item())
        val_accuracy.append(acc)
        val_dice.append(dice)
        
    
    # Results
    print(f'VALIDATION')
    print(f'Epoch: {epoch} | Loss: {np.mean(val_losses):.4f}, Accuracy: {np.mean(val_accuracy):.3f}, Dice: {np.mean(val_dice):.3f}')
    print('########################################################################################')
    total_val_losses.append(np.mean(val_losses))
    total_val_accuracy.append(np.mean(val_accuracy))
    total_val_dice.append(np.mean(val_dice))
        
    # torch.save(model.state_dict(), f'/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto/Primeros_resultados_PSPNet/PSPNet_res101_368_{epoch}.pt')
    torch.save(model.state_dict(), path_res+'/modelo_'+str(epoch)+'.pt')
    
    
    
    resultados = pd.DataFrame(list(zip(total_train_losses, total_val_losses, total_train_dice, total_val_dice,total_train_accuracy, total_val_accuracy)),
                          columns = ['train_loss', 'val_loss', 'train_dice', 'test_dice', 'train_accuracy','val_accuracy'])
    resultados.to_csv(path_res+'/'+'res_table.csv')
    
    

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(list(range(len(total_train_losses))), total_train_losses)
plt.plot(list(range(len(total_train_losses))), total_val_losses)
plt.legend(['Training loss', 'Validation loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

In [ ]:
resultados.head()

# UNET 2

In [ ]:
epochs = 15
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss().to(device)
total_train_losses   = []
total_train_accuracy=[]
total_val_losses=[]
total_val_accuracy=[]
total_train_dice=[]
total_val_dice=[]

In [ ]:
path_res='/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto_yo/ResUNET_mano/Segundo'
for epoch in range(1, epochs+1):
    
    # Train model
    model.train()
    train_losses   = []
    train_accuracy = []
    train_dice       = []
    
    for i, batch in enumerate(dataloader_val):
        img_batch, mask_batch = batch   
        img_batch = img_batch.to(device)
        mask_batch = mask_batch.to(device)

        #Train model
        optimizer.zero_grad()
        output = model(img_batch)
        loss   = criterion(output, mask_batch)
        loss.backward()
        optimizer.step()
        dice = DiceScore(output, mask_batch)
        acc = model_accuracy(output, mask_batch)
        train_losses.append(loss.item())
        train_accuracy.append(acc)
        train_dice.append(dice)
        
    
    # RESULTS
    print(f'TRAINING')
    print(f'Epoch: {epoch} | Loss: {np.mean(train_losses):.4f}, Accuracy: {np.mean(train_accuracy):.3f}, Dice: {np.mean(train_dice):.3f}')        
    total_train_losses.append(np.mean(train_losses))
    total_train_accuracy.append(np.mean(train_accuracy))
    total_train_dice.append(np.mean(train_dice))
    
    
    ######################### Validation of the model
    model.eval()
    val_losses   = []
    val_accuracy = []
    val_dice       = []
    
    for i, batch in enumerate(dataloader_val):
        img_batch, mask_batch = batch
        img_batch = img_batch.to(device)
        mask_batch = mask_batch.to(device)
        with torch.cuda.amp.autocast():
          output = model(img_batch)
          loss   = criterion(output, mask_batch)
        dice = DiceScore(output, mask_batch)
        acc = model_accuracy(output, mask_batch)
        val_losses.append(loss.item())
        val_accuracy.append(acc)
        val_dice.append(dice)
        
    
    # Results
    print(f'VALIDATION')
    print(f'Epoch: {epoch} | Loss: {np.mean(val_losses):.4f}, Accuracy: {np.mean(val_accuracy):.3f}, Dice: {np.mean(val_dice):.3f}')
    print('########################################################################################')
    total_val_losses.append(np.mean(val_losses))
    total_val_accuracy.append(np.mean(val_accuracy))
    total_val_dice.append(np.mean(val_dice))
        
    # torch.save(model.state_dict(), f'/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto/Primeros_resultados_PSPNet/PSPNet_res101_368_{epoch}.pt')
    torch.save(model.state_dict(), path_res+'/modelo_'+str(epoch)+'.pt')
    resultados = pd.DataFrame(list(zip(total_train_losses, total_val_losses, total_train_dice, total_val_dice,total_train_accuracy, total_val_accuracy)),
                          columns = ['train_loss', 'val_loss', 'train_dice', 'test_dice', 'train_accuracy','val_accuracy'])
    resultados.to_csv(path_res+'/'+'res_table.csv')
    


In [ ]:
plt.figure(figsize=(10,8))
plt.plot(list(range(len(total_train_losses))), total_train_losses)
plt.plot(list(range(len(total_train_losses))), total_val_losses)
plt.legend(['Training loss', 'Validation loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

In [ ]:
epochs = 20
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss().to(device)
total_train_losses   = []
total_train_accuracy=[]
total_val_losses=[]
total_val_accuracy=[]
total_train_dice=[]
total_val_dice=[]

In [ ]:
path_res='/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto_yo/ResUNET_mano/Tercero'
for epoch in range(1, epochs+1):
    
    # Train model
    model.train()
    train_losses   = []
    train_accuracy = []
    train_dice       = []
    
    for i, batch in enumerate(dataloader_val):
        img_batch, mask_batch = batch   
        img_batch = img_batch.to(device)
        mask_batch = mask_batch.to(device)

        #Train model
        optimizer.zero_grad()
        output = model(img_batch)
        loss   = criterion(output, mask_batch)
        loss.backward()
        optimizer.step()
        dice = DiceScore(output, mask_batch)
        acc = model_accuracy(output, mask_batch)
        train_losses.append(loss.item())
        train_accuracy.append(acc)
        train_dice.append(dice)
        
    
    # RESULTS
    print(f'TRAINING')
    print(f'Epoch: {epoch} | Loss: {np.mean(train_losses):.4f}, Accuracy: {np.mean(train_accuracy):.3f}, Dice: {np.mean(train_dice):.3f}')        
    total_train_losses.append(np.mean(train_losses))
    total_train_accuracy.append(np.mean(train_accuracy))
    total_train_dice.append(np.mean(train_dice))
    
    
    ######################### Validation of the model
    model.eval()
    val_losses   = []
    val_accuracy = []
    val_dice       = []
    
    for i, batch in enumerate(dataloader_val):
        img_batch, mask_batch = batch
        img_batch = img_batch.to(device)
        mask_batch = mask_batch.to(device)

        
        with torch.cuda.amp.autocast():
          output = model(img_batch)
          loss   = criterion(output, mask_batch)
        dice = DiceScore(output, mask_batch)
        acc = model_accuracy(output, mask_batch)
        val_losses.append(loss.item())
        val_accuracy.append(acc)
        val_dice.append(dice)
        
    
    # Results
    print(f'VALIDATION')
    print(f'Epoch: {epoch} | Loss: {np.mean(val_losses):.4f}, Accuracy: {np.mean(val_accuracy):.3f}, Dice: {np.mean(val_dice):.3f}')
    print('########################################################################################')
    total_val_losses.append(np.mean(val_losses))
    total_val_accuracy.append(np.mean(val_accuracy))
    total_val_dice.append(np.mean(val_dice))
        
    # torch.save(model.state_dict(), f'/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto/Primeros_resultados_PSPNet/PSPNet_res101_368_{epoch}.pt')
    torch.save(model.state_dict(), path_res+'/modelo_'+str(epoch)+'.pt')
    resultados = pd.DataFrame(list(zip(total_train_losses, total_val_losses, total_train_dice, total_val_dice,total_train_accuracy, total_val_accuracy)),
                          columns = ['train_loss', 'val_loss', 'train_dice', 'test_dice', 'train_accuracy','val_accuracy'])
    resultados.to_csv(path_res+'/'+'res_table.csv')

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(list(range(len(total_train_losses))), total_train_losses)
plt.plot(list(range(len(total_train_losses))), total_val_losses)
plt.legend(['Training loss', 'Validation loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

In [ ]:
# even though the epoch 11 has the best accuracy for this net, the dice scrore is super low

# UNET using ResNet as encoder

In [ ]:
!pip install -q -U segmentation-models-pytorch albumentations > /dev/null
import segmentation_models_pytorch as smp

In [ ]:
model = smp.Unet(
    encoder_name = 'resnet34', 
    encoder_weights = 'imagenet', 
    classes = 27, 
    activation = None,
).to(device)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

In [ ]:
epochs = 20
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss().to(device)
total_train_losses   = []
total_train_accuracy=[]
total_val_losses=[]
total_val_accuracy=[]
total_train_dice=[]
total_val_dice=[]

In [ ]:
path_res='/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto_yo/PSPnet_UNET/PSP_Unet_primero'
for epoch in range(1, epochs+1):
    
    # Train model
    model.train()
    train_losses   = []
    train_accuracy = []
    train_dice       = []
    
    for i, batch in enumerate(dataloader_val):
        img_batch, mask_batch = batch   
        img_batch = img_batch.to(device)
        mask_batch = mask_batch.to(device)

        #Train model
        optimizer.zero_grad()
        
        output = model(img_batch)
        loss   = criterion(output, mask_batch)
        loss.backward()
        optimizer.step()        
        dice = DiceScore(output, mask_batch)
        acc = model_accuracy(output, mask_batch)
        train_losses.append(loss.item())
        train_accuracy.append(acc)
        train_dice.append(dice)
        
    
    # RESULTS
    print(f'TRAINING')
    print(f'Epoch: {epoch} | Loss: {np.mean(train_losses):.4f}, Accuracy: {np.mean(train_accuracy):.3f}, Dice: {np.mean(train_dice):.3f}')        
    total_train_losses.append(np.mean(train_losses))
    total_train_accuracy.append(np.mean(train_accuracy))
    total_train_dice.append(np.mean(train_dice))
    
    
    ######################### Validation of the model
    model.eval()
    val_losses   = []
    val_accuracy = []
    val_dice       = []
    
    for i, batch in enumerate(dataloader_val):      
        img_batch, mask_batch = batch
        img_batch = img_batch.to(device)
        mask_batch = mask_batch.to(device)
        with torch.cuda.amp.autocast():
          output = model(img_batch)
          loss   = criterion(output, mask_batch)
        dice = DiceScore(output, mask_batch)
        acc = model_accuracy(output, mask_batch)
        val_losses.append(loss.item())
        val_accuracy.append(acc)
        val_dice.append(dice)
        
    
    # Results
    print(f'VALIDATION')
    print(f'Epoch: {epoch} | Loss: {np.mean(val_losses):.4f}, Accuracy: {np.mean(val_accuracy):.3f}, Dice: {np.mean(val_dice):.3f}')
    print('########################################################################################')
    total_val_losses.append(np.mean(val_losses))
    total_val_accuracy.append(np.mean(val_accuracy))
    total_val_dice.append(np.mean(val_dice))
        
    # torch.save(model.state_dict(), f'/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto/Primeros_resultados_PSPNet/PSPNet_res101_368_{epoch}.pt')
    torch.save(model.state_dict(), path_res+'/modelo_'+str(epoch)+'.pt')
    resultados = pd.DataFrame(list(zip(total_train_losses, total_val_losses, total_train_dice, total_val_dice,total_train_accuracy, total_val_accuracy)),
                          columns = ['train_loss', 'val_loss', 'train_dice', 'test_dice', 'train_accuracy','val_accuracy'])
    resultados.to_csv(path_res+'/'+'res_table.csv')

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(list(range(len(total_train_losses))), total_train_losses)
plt.plot(list(range(len(total_train_losses))), total_val_losses)
plt.legend(['Training loss', 'Validation loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

# **PSPNet using ResNet encoder**

In [ ]:
model = smp.PSPNet(
    encoder_name = 'resnet101', 
    encoder_weights = 'imagenet', 
    classes = 27, 
    activation = None, # could be None for logits or 'softmax2d' for multiclass segmentation
).to(device)

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

In [ ]:
epochs = 25
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss().to(device)
total_train_losses   = []
total_train_accuracy=[]
total_val_losses=[]
total_val_accuracy=[]
total_train_dice=[]
total_val_dice=[]

In [ ]:
path_res='/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto_yo/PSPnet_RESnet'
for epoch in range(1, epochs+1):
    
    # Train model
    model.train()
    train_losses   = []
    train_accuracy = []
    train_dice       = []  
    for i, batch in enumerate(dataloader_val):
        img_batch, mask_batch = batch   
        img_batch = img_batch.to(device)
        mask_batch = mask_batch.to(device)

        #Train model
        optimizer.zero_grad()     
        output = model(img_batch)
        loss   = criterion(output, mask_batch)
        loss.backward()
        optimizer.step()       
        dice = DiceScore(output, mask_batch)
        acc = model_accuracy(output, mask_batch)
        train_losses.append(loss.item())
        train_accuracy.append(acc)
        train_dice.append(dice)
        
    
    # RESULTS
    print(f'TRAINING')
    print(f'Epoch: {epoch} | Loss: {np.mean(train_losses):.4f}, Accuracy: {np.mean(train_accuracy):.3f}, Dice: {np.mean(train_dice):.3f}')        
    total_train_losses.append(np.mean(train_losses))
    total_train_accuracy.append(np.mean(train_accuracy))
    total_train_dice.append(np.mean(train_dice))
    
    
    ######################### Validation of the model
    model.eval()
    val_losses   = []
    val_accuracy = []
    val_dice       = []
    
    for i, batch in enumerate(dataloader_val):   
        img_batch, mask_batch = batch
        img_batch = img_batch.to(device)
        mask_batch = mask_batch.to(device)
      
        with torch.cuda.amp.autocast():
          output = model(img_batch)
          loss   = criterion(output, mask_batch)

        dice = DiceScore(output, mask_batch)
        acc = model_accuracy(output, mask_batch)
        val_losses.append(loss.item())
        val_accuracy.append(acc)
        val_dice.append(dice)
        
    
    # Results
    print(f'VALIDATION')
    print(f'Epoch: {epoch} | Loss: {np.mean(val_losses):.4f}, Accuracy: {np.mean(val_accuracy):.3f}, Dice: {np.mean(val_dice):.3f}')
    print('########################################################################################')
    total_val_losses.append(np.mean(val_losses))
    total_val_accuracy.append(np.mean(val_accuracy))
    total_val_dice.append(np.mean(val_dice))
        
    # torch.save(model.state_dict(), f'/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto/Primeros_resultados_PSPNet/PSPNet_res101_368_{epoch}.pt')
    torch.save(model.state_dict(), path_res+'/modelo_'+str(epoch)+'.pt')
    
    resultados = pd.DataFrame(list(zip(total_train_losses, total_val_losses, total_train_dice, total_val_dice,total_train_accuracy, total_val_accuracy)),
                          columns = ['train_loss', 'val_loss', 'train_dice', 'test_dice', 'train_accuracy','val_accuracy'])
    resultados.to_csv(path_res+'/'+'res_table.csv')

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(list(range(len(total_train_losses))), total_train_losses)
plt.plot(list(range(len(total_train_losses))), total_val_losses)
plt.legend(['Training loss', 'Validation loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

# Format for submission

In [ ]:
# select model
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto/1_Primeros_resultados_PSPNet/PSPNet_res101_368_1.pt'))

<All keys matched successfully>

In [ ]:
# Load test images
test_list = sorted(glob.glob('/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proyecto/TheTensors/test_images/*'))
torch_list = [torch.load(i) for i in test_list]

In [ ]:
model.eval()

# calculate output for each image in test set and save the prediction in new folder
for i in tqdm(range(len(torch_list))):
  img_id = test_list[i].split('/')[-1].split('.')[0]
  img = torch_list[i].unsqueeze(0)
  output = model(img)
  output = torch.argmax(output, dim=1).squeeze(0)
  output = np.uint8(output)
  output = Image.fromarray(output)
  output.save(f'/content/drive/MyDrive/Colab Notebooks/Deep Learning/Final/{img_id}.png')

In [ ]:
import tarfile

tar = tarfile.open("submission.tar", "w")

for root, dir, files in os.walk('/content/drive/MyDrive/Colab Notebooks/Deep Learning/Final'):
    for  file in files:
        fullpath = os.path.join(root, file)
        tar.add(fullpath, arcname=file)

tar.close()